In [1]:
## Importação das Bibiliotecas

In [29]:
from pyspark.sql import SparkSession
import pandas as pd
from sqlalchemy import create_engine
import os
from pyspark.sql.functions import col, count, sum
import pymssql
#import pyodbc


## Criando uma SparkSession

In [39]:
#Usando a Imagem Docker
#spark = SparkSession.builder.remote("sc://spark-connect").getOrCreate()

In [30]:
# Crie um SparkSession
spark = SparkSession.builder.appName("LerDadosEnem").getOrCreate()

## Processo de Leitura dos arquivos CSV

In [3]:
#Path dos arquivos usando Imagem do docker

#caminho_arquivo_2022 = '/opt/datasets/ENEM/2022/microdados_enem_2022/DADOS/MICRODADOS_ENEM_2022.csv'
#caminho_arquivo_2021 = '/opt/datasets/ENEM/2021/microdados_enem_2021/DADOS/MICRODADOS_ENEM_2021.csv'
#caminho_arquivo_2020 = '/opt/datasets/ENEM/2020/microdados_enem_2020/DADOS/MICRODADOS_ENEM_2020.csv'
#caminho_arquivo_2019 = '/opt/datasets/ENEM/2019/microdados_enem_2019/DADOS/MICRODADOS_ENEM_2019.csv'

In [4]:
#Path dos arquivos

caminho_arquivo_2022 = r'E:\Estudos\SQL\Datasets\ENEM\2022\microdados_enem_2022\DADOS\MICRODADOS_ENEM_2022.csv'
caminho_arquivo_2021 = r'E:\Estudos\SQL\Datasets\ENEM\2021\microdados_enem_2021\DADOS\MICRODADOS_ENEM_2021.csv'
caminho_arquivo_2020 = r'E:\Estudos\SQL\Datasets\ENEM\2020\microdados_enem_2020\DADOS\MICRODADOS_ENEM_2020.csv'
caminho_arquivo_2019 = r'E:\Estudos\SQL\Datasets\ENEM\2019\microdados_enem_2019\DADOS\MICRODADOS_ENEM_2019.csv'

In [5]:
#Função Paa ler os arquivos CSV
def lerCSV(spark, caminho_arquivo):
    df = spark.read.csv(caminho_arquivo, sep=';', header=True, inferSchema=True, encoding='ISO-8859-1')
    #colunas_selecionadas = df.columns[12:19]
    #return df.select(colunas_selecionadas)
    return df


## Criação dos Dataframes e Unificaçao

In [31]:
##Chama função quer ler o arquivo CSV e Transforma para DF.SPARK

dfEscola22 = lerCSV(spark, caminho_arquivo_2022)
#dfEscola21 = lerCSV(spark, caminho_arquivo_2021)
#dfEscola20 = lerCSV(spark, caminho_arquivo_2020)
#dfEscola19 = lerCSV(spark, caminho_arquivo_2019)


In [7]:
#Unir os DF em um só 

#dfDadosGerais = dfEscola22.union(dfEscola21).union(dfEscola20).union(dfEscola19)
dfDadosGerais = dfEscola22


In [8]:
dfPreencher = dfDadosGerais.fillna(-2)  # Preenche com -2(sem ref), por exemplo
#dfPreencher.cache()

## Criação das Dimes

### Dime Municipios

In [9]:
dfDimeMunicipio = dfPreencher.select(
    col("CO_MUNICIPIO_ESC").alias("MUNICIPIO_ID"),
    col("NO_MUNICIPIO_ESC").alias("MUNICIPIO_DESCRICAO"),
    col("CO_UF_ESC").alias("UF_ID")
).distinct()

### Dime UF

In [61]:
dfUF = dfPreencher.select(
    col("CO_UF_ESC").alias("UF_ID"),
    col("SG_UF_ESC").alias("UF_SIGLA")
).distinct()

dfUF.createOrReplaceTempView("TEMP_UF")

### Dicionário de Siglas UF dos Estados Brasileiros e sua descrição

In [62]:
dfUFDesc = spark.sql("""
        SELECT T.*,A.UF_DESCRICAO FROM(
        SELECT 'RO' AS UF_ID, 'Rondônia' AS UF_DESCRICAO
        UNION ALL
        SELECT 'AC', 'Acre'
        UNION ALL
        SELECT 'AM', 'Amazonas'
        UNION ALL
        SELECT 'RR', 'Roraima'
        UNION ALL
        SELECT 'PA', 'Pará'
        UNION ALL
        SELECT 'AP', 'Amapá'
        UNION ALL
        SELECT 'TO', 'Tocantins'
        UNION ALL
        SELECT 'MA', 'Maranhão'
        UNION ALL
        SELECT 'PI', 'Piauí'
        UNION ALL
        SELECT 'CE', 'Ceará'
        UNION ALL
        SELECT 'RN', 'Rio Grande do Norte'
        UNION ALL
        SELECT 'PB', 'Paraíba'
        UNION ALL
        SELECT 'PE', 'Pernambuco'
        UNION ALL
        SELECT 'AL', 'Alagoas'
        UNION ALL
        SELECT 'SE', 'Sergipe'
        UNION ALL
        SELECT 'BA', 'Bahia'
        UNION ALL
        SELECT 'MG', 'Minas Gerais'
        UNION ALL
        SELECT 'ES', 'Espírito Santo'
        UNION ALL
        SELECT 'RJ', 'Rio de Janeiro'
        UNION ALL
        SELECT 'SP', 'São Paulo'
        UNION ALL
        SELECT 'PR', 'Paraná'
        UNION ALL
        SELECT 'SC', 'Santa Catarina'
        UNION ALL
        SELECT 'RS', 'Rio Grande do Sul'
        UNION ALL
        SELECT 'MS', 'Mato Grosso do Sul'
        UNION ALL
        SELECT 'MT', 'Mato Grosso'
        UNION ALL
        SELECT 'GO', 'Goiás'
        UNION ALL
        SELECT 'DF', 'Distrito Federal'
        UNION ALL
        SELECT '-2', 'Sem Referência') A
        INNER JOIN TEMP_UF T
        ON A.UF_ID = T.UF_SIGLA
""")

In [64]:
dfUFDesc.show()

+-----+--------+-------------------+
|UF_ID|UF_SIGLA|       UF_DESCRICAO|
+-----+--------+-------------------+
|   25|      PB|            Paraíba|
|   43|      RS|  Rio Grande do Sul|
|   23|      CE|              Ceará|
|   31|      MG|       Minas Gerais|
|   52|      GO|              Goiás|
|   13|      AM|           Amazonas|
|   35|      SP|          São Paulo|
|   12|      AC|               Acre|
|   26|      PE|         Pernambuco|
|   41|      PR|             Paraná|
|   24|      RN|Rio Grande do Norte|
|   27|      AL|            Alagoas|
|   33|      RJ|     Rio de Janeiro|
|   29|      BA|              Bahia|
|   15|      PA|               Pará|
|   51|      MT|        Mato Grosso|
|   21|      MA|           Maranhão|
|   16|      AP|              Amapá|
|   22|      PI|              Piauí|
|   28|      SE|            Sergipe|
+-----+--------+-------------------+
only showing top 20 rows



In [27]:
dimeManualCorRacao = r'C:\Users\leoba\Documents\Enem\Dataset\Dimes\Dime_Cor_Raca.csv'
dimeManualEscola = r'C:\Users\leoba\Documents\Enem\Dataset\Dimes\Dime_Escola.csv'
dimeManualEstadoCivil = r'C:\Users\leoba\Documents\Enem\Dataset\Dimes\Dime_Estado_Civil.csv'
dimeManualFaixaEtaria = r'C:\Users\leoba\Documents\Enem\Dataset\Dimes\Dime_Faixa_Etaria.csv'
dimeManualFaixaRendaMensal = r'C:\Users\leoba\Documents\Enem\Dataset\Dimes\Dime_Faixa_Renda_Mensal.csv'
dimeManualNacionalidade = r'C:\Users\leoba\Documents\Enem\Dataset\Dimes\Dime_Nacionalidade.csv'
dimeManualSexo = r'C:\Users\leoba\Documents\Enem\Dataset\Dimes\Dime_Sexo.csv'
dimeManualSituacaoEscolaridade = r'C:\Users\leoba\Documents\Enem\Dataset\Dimes\Dime_Situacao_Escolaridade.csv'


In [28]:
##Chama função quer ler o arquivo CSV e Transforma para DF.SPARK
dfDimeCorRaca = lerCSV(spark, dimeManualCorRacao)
dfDimeEscola = lerCSV(spark, dimeManualEscola)
dfDimeEstadoCivil = lerCSV(spark,dimeManualEstadoCivil)
dfDimeFaixaEtaria = lerCSV(spark,dimeManualFaixaEtaria)
dfDimeFaixaRendaMensal = lerCSV(spark,dimeManualFaixaRendaMensal)
dfDimeNacionalidade = lerCSV(spark,dimeManualNacionalidade)
dfDimeSexo = lerCSV(spark,dimeManualSexo)
dfSituacaoEscolaridade = lerCSV(spark,dimeManualSituacaoEscolaridade)


## Criação da Fato Enem

In [69]:
dfPreencherDistinct = dfPreencher.select(
    col("NU_INSCRICAO").alias("NU_INSCRICAO"),
    col("NU_ANO").alias("ANO_EDICAO"),
    col("TP_ANO_CONCLUIU").alias("ANO_CONCLUSAO"),
    col("TP_SEXO").alias("SEXO_CODIGO"),
    col("CO_MUNICIPIO_ESC").alias("MUNICIPIO_CODIGO"),
    col("TP_ESTADO_CIVIL").alias("ESTADO_CIVIL_ID"),
    col("TP_COR_RACA").alias("COR_RACA_ID"),
    col("TP_NACIONALIDADE").alias("NACIONALIDADE_ID"),
    col("TP_ST_CONCLUSAO").alias("SITUACAO_ESCOLARIDADE_ID"),
    col("TP_ESCOLA").alias("ESCOLA_ID"),
    col("Q006").alias("FAIXA_RENDA_MENSAL_ID"),
    col("TP_FAIXA_ETARIA").alias("FAIXA_ETARIA_ID"),
    col("NU_NOTA_CN").alias("NOTA_CIENCIA_DA_NATUREZA"),
    col("NU_NOTA_CH").alias("NOTA_CIENCIA_DA_HUMANA"),
    col("NU_NOTA_LC").alias("NOTA_LINGUAGEM_CODIGO"),
    col("NU_NOTA_MT").alias("NOTA_MATEMATICA")
).distinct()
#dfPreencherDistinct.cache()

In [70]:
dfAgregate = dfPreencherDistinct.select(
    "ANO_EDICAO",
    "ANO_CONCLUSAO",
    "SEXO_CODIGO",
    "MUNICIPIO_CODIGO",
    "ESTADO_CIVIL_ID",
    "COR_RACA_ID",
    "NACIONALIDADE_ID",
    "SITUACAO_ESCOLARIDADE_ID",
    "ESCOLA_ID",
    "FAIXA_RENDA_MENSAL_ID",
    "FAIXA_ETARIA_ID",
    "NU_INSCRICAO",  # Adicionando essa coluna para o count
    "NOTA_CIENCIA_DA_NATUREZA",
    "NOTA_CIENCIA_DA_HUMANA",
    "NOTA_LINGUAGEM_CODIGO",
    "NOTA_MATEMATICA"
)

# Agregação após o agrupamento
dfAgregate = dfAgregate.groupBy(
    "ANO_EDICAO",
    "ANO_CONCLUSAO",
    "SEXO_CODIGO",
    "MUNICIPIO_CODIGO",
    "ESTADO_CIVIL_ID",
    "COR_RACA_ID",
    "NACIONALIDADE_ID",
    "SITUACAO_ESCOLARIDADE_ID",
    "ESCOLA_ID",
    "FAIXA_RENDA_MENSAL_ID",
    "FAIXA_ETARIA_ID"
).agg(
    count("NU_INSCRICAO").alias("QTD"),
    sum("NOTA_CIENCIA_DA_NATUREZA").alias("NOTA_CIENCIA_DA_NATUREZA"),
    sum("NOTA_CIENCIA_DA_HUMANA").alias("NOTA_CIENCIA_DA_HUMANA"),
    sum("NOTA_LINGUAGEM_CODIGO").alias("NOTA_LINGUAGEM_CODIGO"),
    sum("NOTA_MATEMATICA").alias("NOTA_MATEMATICA")
)

dfAgregate.cache()

DataFrame[ANO_EDICAO: int, ANO_CONCLUSAO: int, SEXO_CODIGO: string, MUNICIPIO_CODIGO: int, ESTADO_CIVIL_ID: int, COR_RACA_ID: int, NACIONALIDADE_ID: int, SITUACAO_ESCOLARIDADE_ID: int, ESCOLA_ID: int, FAIXA_RENDA_MENSAL_ID: string, FAIXA_ETARIA_ID: int, QTD: bigint, NOTA_CIENCIA_DA_NATUREZA: double, NOTA_CIENCIA_DA_HUMANA: double, NOTA_LINGUAGEM_CODIGO: double, NOTA_MATEMATICA: double]

In [11]:
dfAgregate.schema

StructType([StructField('ANO_EDICAO', IntegerType(), True), StructField('ANO_CONCLUSAO', IntegerType(), True), StructField('SEXO_CODIGO', StringType(), True), StructField('MUNICIPIO_CODIGO', IntegerType(), True), StructField('ESTADO_CIVIL_ID', IntegerType(), True), StructField('COR_RACA_ID', IntegerType(), True), StructField('NACIONALIDADE_ID', IntegerType(), True), StructField('SITUACAO_ESCOLARIDADE_ID', IntegerType(), True), StructField('ESCOLA_ID', IntegerType(), True), StructField('FAIXA_RENDA_MENSAL_ID', StringType(), True), StructField('FAIXA_ETARIA_ID', IntegerType(), True), StructField('QTD', LongType(), False), StructField('NOTA_CIENCIA_DA_NATUREZA', DoubleType(), True), StructField('NOTA_CIENCIA_DA_HUMANA', DoubleType(), True), StructField('NOTA_LINGUAGEM_CODIGO', DoubleType(), True), StructField('NOTA_MATEMATICA', DoubleType(), True)])

# Salvar em SQL 
Salvar os dados em uma tabela SQL


## Parametrizando a conexão do Banco SQL SERVER Usando JDBC

In [66]:
server_name = "localhost"
database_name = "DockerEnem"
username = "sa"
password = "bi@123456"

In [67]:
# Conectando ao banco de dados
conn = pymssql.connect(server=server_name, database=database_name, user=username, password=password,port='1434')
# Criando um cursor
cursor = conn.cursor()

In [73]:
# Nome da tabela a ser excluída
table_name = "ODS_ENEM"

#obter Schema do dfAgregate 
df_schema = dfAgregate.schema

# Gerando as definições de coluna para a tabela
columns_definitions = [f"{field.name} {field.dataType}" for field in df_schema.fields]

# Mapeamento de tipos de dados do Spark para tipos de dados do SQL Server
data_type_mapping = {
    "IntegerType": "INT",
    "StringType": "NVARCHAR(255)",  # Você pode ajustar o tamanho conforme necessário
    "DoubleType": "FLOAT",
    "LongType": "BIGINT"
}

# Gerando as definições de coluna para a tabela com os tipos de dados corretos
columns_definitions = [f"{field.name} {data_type_mapping.get(str(field.dataType), 'NVARCHAR(255)')}" for field in df_schema.fields]
columns_update = [f"{field.name}" for field in df_schema.fields]


# Combinando as definições de coluna em uma string
columns_str = ",\n".join(columns_definitions)
columns_update_str =  ",\n".join(columns_update)

In [71]:
# Dropando a tabela se ela já existir
drop_table_query = """
IF OBJECT_ID('ODS_ENEM', 'U') IS NOT NULL
    DROP TABLE ODS_ENEM
"""
cursor.execute(drop_table_query)
conn.commit()  # Comitar após dropar a tabela


In [62]:
# Criando o comando CREATE TABLE
create_table_query = f"""
CREATE TABLE {table_name} (
{columns_str}
)
"""
# Executando o comando SQL para criar a tabela
cursor.execute(create_table_query)
conn.commit()  # Comitar após criar a tabela


In [63]:
rows = dfAgregate.collect()
#print(rows)

In [ ]:
insert_query = f"""
    INSERT INTO ODS_ENEM ({columsn_update})
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""
write_data = tuple(map(tuple, rows))
cursor.executemany(insert_query, write_data)
conn.commit()


In [66]:
def chunks(data, chunk_size):
    """Divide a data list em pedaços de tamanho chunk_size."""
    for i in range(0, len(data), chunk_size):
        yield data[i:i + chunk_size]

def insert(query, data, chunk=999):
    query = query.lower()
    insert_q, values_q = query.split('values')
    insert_q += 'values'  # Adiciona 'values' para manter a query SQL correta após o split

    try:
        for chunk_data in chunks(data, chunk):
            flat_list = [item for sublist in chunk_data for item in sublist]
            chunk_query = insert_q + ','.join([values_q] * len(chunk_data))
            cursor.execute(chunk_query, tuple(flat_list))
            conn.commit()  # Comitar após cada chunk
    except pymssql.OperationalError as e:
        print(f"OperationalError: {e}")
        conn.rollback()
    except Exception as e:
        print(f"An error occurred: {e}")
        conn.rollback()

In [71]:
# Definindo a query de inserção
insert_query = f"""
    INSERT INTO {table_name} ({columns_update_str})
    VALUES ({', '.join(['%s'] * len(df_schema.fields))})
"""
# Convertendo os dados para tuplas
write_data = [tuple(row) for row in rows]

In [72]:

insert(insert_query, write_data)

In [73]:
# Fechando cursor e conexão
cursor.close()
conn.close()